# K-means Convex-hull Clusters
*************
Overview
1. Using a clustering method called K-means, write an algorithm to cluster a set of 3D points
2. Write an algorithm to construct the convex hull of each cluster
*********
K-means Clustering Algorithm
1. Develop a k-means algorithm",
    * choose x number of cluster centers and choose x random points to be the centers
    * assign points to closest center
    * re-evaluate cluster centers using the equation 
    $$\mu_i^{'} = \frac{1}{C_i} \sum_{x \in C_i}x$$
    * re-assign points closest to cluster centers
    * terminate iteration process once no new point assignments
2. Implement a function that computes the sum of squared errors (SSE) given by the function: \n",
    $$SSE = \sum_{i=1}^K \sum_{x \in C_i} ||x - \mu_i||_2^2$$ where $C_i$ is cluster $i$ and $\mu_i$ is the center of cluster $C_i$.
***********
3D Convex Hull Algorithm
* Have user input a number n to generate a random set of points in R^3
* Determine which sets of 3 points from the pointset form a plane that is on the "outside" of the pointset. This is done by seeing if all remaining points lie to the same side of the plane. This "side" is defined by a normal vector relative to the plane
* Plot each triangular plane defined by the sets of 3 points to form a 3D convex shape around the pointset
* Show which points do not lie on the convex hull inside of the shape

In [ ]:
#imports and point selection

import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab as mylab
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import math
import random

#Generate array list of number of points specified by user
x = int(input('Please input the number of points you want in the pointset:')) #user input number of points in pointset S
matrix = np.random.choice(range(4*x), (x,3), replace=False)
clus_num = int(input("How many cluster hulls do you want? *Note: if a cluster has less than 4 points, it will be eliminated "))

In [ ]:
#Divide all points into separate clusters
clus = int(len(matrix)/clus_num)
center_num = int(len(matrix)/clus) #bigger denominator means less clusters, but more points in each cluster

index = list(np.random.choice(matrix.shape[0], center_num, replace = False)) #choose x number of random indices from matrix (based on user input)
center_start = [] 
for i in index: #append the elements of matrix to center_start from index
    center_start.append(matrix[i].tolist())
center_start = np.asarray(center_start)

#create a dictionary for cluster centers and clusters. Assign points to clusters based on initial cluster centers

cluster_centers = {}
for i in index:
    cluster_centers[i] = matrix[i].tolist()

clusters = {cluster_index:[] for cluster_index in index} #a dictionary where a list of points closest to a cluster can be tied to cluster center

for i in matrix: #for every point in dataset
    distances = {} #dictionary where key is cluster center and the value is the distance between the cluster center and point i
    for j in index: #for each element in the list of indices of cluster centers
        distance = math.sqrt((i[0] - matrix[j][0])**2 + (i[1] - matrix[j][1])**2 + (i[2] - matrix[j][2])**2) #euclidean distance metric
        distances[j] = distance #adds key j and value distance to distances dictionary
    minimum_key = min(distances.keys(), key=(lambda k: distances[k])) #finds the key with the minimum value
    minimum_value = distances.get(minimum_key) #finds the value associated with minimum key
    clusters[minimum_key].append(list(i)) #append point i to the cluster center it is closest to

#print ('initial cluster centers: \n\n', cluster_centers, '\n\ninitial clusters: \n')
#clusters

#function for computing cluster centers

def center(points): #have points be a numpy 2d array
    total_sum_x = 0 #sum of all x coordinates
    total_sum_y = 0 #sum of all y coordinates
    total_sum_z = 0 #sum of all z coordinates
    for i in points:
        x = i[0]
        y = i[1]
        z = i[2]
        total_sum_x += x
        total_sum_y += y
        total_sum_z += z
    avg_sum_x = (1/len(points))*total_sum_x #average of all x values
    avg_sum_y = (1/len(points))*total_sum_y #average of all y values
    avg_sum_z = (1/len(points))*total_sum_z #average of all z values
    return [avg_sum_x, avg_sum_y, avg_sum_z]

#Compute new center of each cluster. Reassign points to new clusters based on cluster centers. Continue this process until no more changes.

while 1 == True: #always continue while loop until broken
    terminate = 0 #if this is not 0 we terminate process
    for cen in cluster_centers: #compute new center for each cluster
        cluster_centers[cen] = center(clusters[cen])
    sort = sorted(clusters.keys()) #dictionaries are unsorted. This sorts clusters so we can enumerate it
    for i,v in enumerate(sort): #i is the index of clusters element, v is the element
        new_sort = sort.copy()
        new_sort.remove(v) #sorted clusters without the cluster we are looking at
        for value in clusters[v]: #for each point in the cluster we are looking at
            distance_old = math.sqrt((cluster_centers[sort[i]][0] - value[0])**2 + (cluster_centers[sort[i]][1] - value[1])**2 + (cluster_centers[sort[i]][2] - value[2])**2) #find distance between cluster point and its center
            distance_other = {} #holds the distances between the point we are looking at and the other cluster centers
            for w in new_sort: #we can look at the other cluster centers
                distance = math.sqrt((cluster_centers[w][0] - value[0])**2 + (cluster_centers[w][1] - value[1])**2 + (cluster_centers[w][2] - value[2])**2) #distance between point we are looking at and other cluster centers
                distance_other[w] = distance #add to distance_other dictionary
                minimum_key = min(distance_other.keys(), key=(lambda k: distance_other[k])) #finds dictionary key associate with smallest distance value
                distance_new = distance_other.get(minimum_key) #finds value of smallest distance key
            if distance_new < distance_old: #if the point we are looking at is closer to another cluster center, move it to that cluster
                clusters[minimum_key].append(value)
                clusters[v].remove(value)
                terminate += 1 #if this condition is not met, terminate=0 and the while loop breaks
    if terminate == False:
        break

key_list = []
for i in clusters:
    if len(clusters[i]) <= 3:
        key_list.append(i)
for j in key_list:
    del clusters[j]
        
clusters

In [ ]:
#Convex Hull functions 

#index rows of ndarray list
def indx(List, element): 
    index = 0
    for i in List:
        if i[0] == element[0] and i[1] == element[1] and i[2] == element[2]: #if x-coor == x-coor, y-coor == y-coor, z-coor == z-coor
            return index
        index += 1

#Create a vector between two points
def create_vector(p1,p2): #vector points from p1 to p2
    vector = (p2[0]-p1[0], p2[1]-p1[1], p2[2]-p1[2])
    return vector

#Check if a point lies above a plane defined by 3 points
def above_plane(p1,p2,p3,t1): #p1,p2,p3 define our plane
    v1 = [p2[0]-p1[0],p2[1]-p1[1],p2[2]-p1[2]] #a vector in our plane
    v2 = [p3[0]-p1[0],p3[1]-p1[1],p3[2]-p1[2]] #another vector in our plane

    #a,b,c defines a vector tangent to our plane
    a = v1[1]*v2[2] - v1[2]*v2[1] 
    b = -(v1[0]*v2[2] - v1[2]*v2[0])
    c = v1[0]*v2[1] - v1[1]*v2[0]

    #coordinates of t1, which we are checking is above plane
    x = t1[0] 
    y = t1[1]
    z = t1[2]

    plane_eq = a*(x-p1[0]) + b*(y-p1[1]) + c*(z-p1[2]) #equation of plane 

    n = [a,b,c] #normal vector to plane
    p1t1 = [t1[0]-p1[0],t1[1]-p1[1],t1[2]-p1[2]] #vector from plane to t1

    dot = n[0]*p1t1[0] + n[1]*p1t1[1] + n[2]*p1t1[2] #dot product of n and p1t1

    return dot >= 0 #return True if point is above plane

#Check if a point lies above a plane defined by 3 points
def below_plane(p1,p2,p3,t1): #p1,p2,p3 define our plane
    v1 = [p2[0]-p1[0],p2[1]-p1[1],p2[2]-p1[2]] #a vector in our plane
    v2 = [p3[0]-p1[0],p3[1]-p1[1],p3[2]-p1[2]] #another vector in our plane

    #a,b,c defines a vector tangent to our plane
    a = v1[1]*v2[2] - v1[2]*v2[1] 
    b = -(v1[0]*v2[2] - v1[2]*v2[0])
    c = v1[0]*v2[1] - v1[1]*v2[0]

    #coordinates of t1, which we are checking is below plane
    x = t1[0] 
    y = t1[1]
    z = t1[2]

    plane_eq = a*(x-p1[0]) + b*(y-p1[1]) + c*(z-p1[2]) #equation of plane 

    n = [a,b,c] #normal vector to plane
    p1t1 = [t1[0]-p1[0],t1[1]-p1[1],t1[2]-p1[2]] #vector from plane to t1

    dot = n[0]*p1t1[0] + n[1]*p1t1[1] + n[2]*p1t1[2] #dot product of n and p1t1

    return dot <= 0 #return True if point is below plane

#Create convex hull for each cluster pointset

for i in clusters:
    
    S = clusters[i]

    hull_pts = []

    print('Hull faces: ')

    for i in S:
        for j in S[indx(S, i)+1:]: #for every element after i
            for k in S[indx(S, j)+1:]: #for every element after j
                above = 0
                below = 0
                for m in S:
                    if m!=i and m!=j and m!=k: #for every element that is not i,j,k
                        if above_plane(i,j,k,m) == True: 
                            above += 1
                            #print('above')
                        if below_plane(i,j,k,m) == True:
                            below += 1
                            #print('below')
                        if above == len(S)-3 or below == len(S)-3: #if the rest of the points lie on the same side of plane i,j,k
                            hull_pts.append(i), hull_pts.append(j), hull_pts.append(k)
                            print(i,j,k)

    print('The n-gon has ',int(len(hull_pts)/3), ' faces\n')

    S2 = S.copy()
    for i in S2:
        if i in hull_pts:
            S2.remove(i)

    S3 = np.asarray(S2) #list of non-hull points
    U = np.asarray(hull_pts) #list of hull points

    #x,y,z of non-hull points
    sx = S3[:,0]
    sy = S3[:,1]
    sz = S3[:,2]

    #x,y,z of hull points
    ux = U[:,0]
    uy = U[:,1]
    uz = U[:,2]

    first = [] #first point of each triangular plane
    second = [] #second point of each triangular plane
    third = [] #third point of each triangular plane

    for i in range(len(U)//3):
        first.append(3*i)
        second.append(3*i+1)
        third.append(3*i+2)

    faces = np.array([first,second,third])
    mylab.triangular_mesh(ux,uy,uz, faces.T, opacity=.5, color=(random.uniform(0,1),random.uniform(0,1),random.uniform(0,1))) #graphs triangular faces
    mylab.triangular_mesh(ux,uy,uz, faces.T, representation='wireframe', color=(0,0,0)) #graphs wire frame of faces 
    mylab.points3d(ux,uy,uz, scale_factor=8) #graphs hull points #sf=2.5
    mylab.points3d(sx,sy,sz, color=(1,0,0), scale_factor=6)#, opacity=.4) #graphs non-hull points #sf=2
    
mylab.show()